In [983]:
from IPython.display import HTML
display(HTML("<head><link rel='stylesheet' type='text/css' href='./../../static/custom.css'></head>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

In [984]:
import numpy as np

from bqplot import *
import bqplot as bq
import bqplot.marks as bqm
import bqplot.scales as bqs
import bqplot.axes as bqa

import ipywidgets as widgets

In [985]:
def get_n(z, dens, mass_mol):
    n = z*Na * dens/mass_mol
    return n

In [986]:
def get_Ef(n):
    Ef = h**2/(2*me)*(3*pi**2*n)**(2.0/3.0)
    return Ef

In [987]:
def get_mu(Ef,T):
    mu = Ef*(1.0-1.0/3.0*(pi*kb*T/2.0/Ef)**2)
    return mu

In [988]:
def update_metal(change):
    metal = metal_dropdown.value
    rho = metal_densities[metal]
    mass = molar_masses[metal]
    z = valence_numbers[metal]
    n = get_n(z, rho, mass)
    Ef = get_Ef(n)
    
    fermi_energy_text.value = '%.2f' % Ef

In [989]:
def update_temp(change):
    T = T_slider.value
    mu = get_mu(Ef,T)

  
    y_values = np.empty(pts)
    for i in range(pts):
        eps = x_values[i]*mu
        y_values[i] = 1.0/(e**((eps-mu)/kb/T)+1.0)

    lines_001.x = x_values
    lines_001.y = y_values
    
    point_002.x = [T]
    point_002.y = [mu]
    
    mu_text.value ='%.2f' % (mu/Ef)

In [990]:
pts = 2000
dim = 3
T = 2.0
T_min = 1.0
T_max = 20.0
h = 6.5821e-16 #Planck's reduced constant in eV*s
c = 3.0e8 # Speed of Light in m/s
me = 510998.9/c**2 # Electron mass in eV/c**2
Na = 6.0221e23
kb = 8.617e-1 # Boltzmann's in eV/(10**4K)
pi =  np.pi
e = np.e

# Metal data:
metal_labels = ["Sodioa", "Aluminioa", "Burdina", "Kobrea"] 
metal_densities = [968.0, 2700.0, 7874.0, 8960.0] # In kg/m**3
molar_masses = [0.0229, 0.026, 0.0558, 0.0635] # In Kg
valence_numbers = [1, 3, 2, 1]

# Test calculation
metal = 1
rho = metal_densities[metal]
mass = molar_masses[metal]
z = valence_numbers[metal]
n = get_n(z, rho, mass)
Ef = get_Ef(n)
mu = get_mu(Ef,T)

x_values = np.linspace(0.0,2.0,pts)
y_values = np.empty(pts)
for i in range(pts):
    E = x_values[i]*mu
    y_values[i] = 1.0/(e**((E-mu)/kb/T)+1.0)

In [991]:
# Create the Figure
fig_121_001 = bq.Figure(title='Fermi-Dirac banaketa',
                marks=[],
                axes=[],
                animation_duration=0,
                legend_location='top-left',
                legend_style= {'fill': 'white', 'stroke': 'grey'},
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                layout = widgets.Layout(width='50%'),
                toolbar = True,
    )

fig_121_002 = bq.Figure(title='Potentzial kimikoa',
                marks=[],
                axes=[],
                animation_duration=0,
                legend_location='top-left',
                legend_style= {'fill': 'white', 'stroke': 'grey'},
                background_style= {'fill': 'white',  'stroke': 'black'},
                fig_margin=dict(top=70, bottom=60, left=80, right=30),
                layout = widgets.Layout(width='50%'),
                toolbar = True,
    )

scale_x_001 = bqs.LinearScale(min = 0.0, max = 2.0)
scale_y_001 = bqs.LinearScale(min = 0.0, max = 1.0)

axis_x_001 = bqa.Axis(scale=scale_x_001,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                #tick_values = np.linspace(p_min, p_max, 7),
                num_ticks=3,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='e/mu',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y_001 = bqa.Axis(
                scale=scale_y_001,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=5,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='n',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_121_001.axes = [axis_x_001,axis_y_001]

lines_001 = bqm.Lines(
                x = x_values, 
                y = y_values, 
                scales = {'x': scale_x_001, 'y': scale_y_001}, 
                opacities = [1.0],
                visible = True, #True, #t == '1.00',
                #colors = colors,
                #labels = labels,
                display_legend = False
)

fig_121_001.marks = [lines_001]

T_values = np.linspace(T_min, T_max, pts)
mu_values = np.empty(pts)
for i in range(pts):
    mu_values[i] = get_mu(Ef, T_values[i])

scale_x_002 = bqs.LinearScale(min = T_min, max = T_max)
scale_y_002 = bqs.LinearScale(min = np.min(mu_values), max = np.max(mu_values))

axis_x_002 = bqa.Axis(scale=scale_x_002,
                tick_format='.0f',#'0.2f',
                tick_style={'font-size': '15px'},
                #tick_values = np.linspace(p_min, p_max, 7),
                num_ticks=3,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                label='T (x10^4 K)',
                label_location='middle',
                label_style={'stroke': 'black', 'default-size': 35},
                label_offset='50px')

axis_y_002 = bqa.Axis(
                scale=scale_y_002,
                tick_format='.1f',#'0.2f',
                tick_style={'font-size': '15px'},
                num_ticks=4,
                grid_lines = 'none',
                grid_color = '#8e8e8e', 
                orientation='vertical',
                label='mu/Ef',
                label_location='middle',
                label_style={'stroke': 'red', 'default_size': 35},
                label_offset='50px')

fig_121_002.axes = [axis_x_002,axis_y_002]


lines_002 = bqm.Lines(
                x = T_values, 
                y = mu_values, 
                scales = {'x': scale_x_002, 'y': scale_y_002}, 
                opacities = [1.0],
                visible = True, #True, #t == '1.00',
                #colors = colors,
                #labels = labels,
                display_legend = False
)

point_002 = bqm.Scatter(
                x = [T], 
                y = [mu], 
                scales = {'x': scale_x_002, 'y': scale_y_002}, 
                opacities = [1.0],
                visible = True, #True, #t == '1.00',
                colors = ['red'],
                #labels = labels,
                display_legend = False
)


fig_121_002.marks = [lines_002, point_002]

T_slider = widgets.FloatSlider(
    min=T_min,
    max=T_max,
    step=(T_max-T_min)/pts,
    value=T,
    description='T',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout = widgets.Layout(width='90%'),
)

T_slider.observe(update_temp, 'value')

metal_dropdown = widgets.Dropdown(
    options=[('Sodioa', 0), ('Aluminioa', 1), ('Burdina', 2), ('Kobrea', 3)],
    value=metal,
    description='Gasa:',
    disabled=False,
)

metal_dropdown.observe(update_metal, 'value') 


# Output values

fermi_energy_text = widgets.Label(value='%.2f' % Ef)
mu_text = widgets.Label(value='%.2f' % (mu/Ef))




# Layout

left_block_121_000 = widgets.VBox([], layout=widgets.Layout(width='20%', align_self='center', align_items='center'))
left_block_121_000.children = [metal_dropdown,
                               widgets.HBox([T_slider, widgets.Label(value='$x10^4 K$')], layout=widgets.Layout(width='100%'))]

center_block_121_000 = widgets.HBox([], layout=widgets.Layout(width='60%', align_self='center', align_items='center'))
center_block_121_000.children = [fig_121_001, fig_121_002]

right_block_121_000 = widgets.VBox([], layout=widgets.Layout(width='20%', align_self='center', align_items='center'))
right_block_121_000.children = [widgets.HBox([widgets.Label(value='$\epsilon_F=$'),fermi_energy_text,widgets.Label(value='$eV$')]),
                                widgets.HBox([widgets.Label(value='$\mu=$'),mu_text,widgets.Label(value='$\epsilon_F$')])]

main_block_121_000 = widgets.HBox([], layout=widgets.Layout(width='100%', align_self='center', align_items='center'))
main_block_121_000.children = [left_block_121_000, center_block_121_000, right_block_121_000]

main_block_121_000